<a href="https://colab.research.google.com/github/RudyMartin/dsai-2024/blob/main/MVPS/Camp-Rock-Paper-Scissors/team_GG_AI/gestures_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup project directories and configuration
import os
from google.colab import drive
import datetime

# Record the start time for performance evaluation
start_time = datetime.datetime.now()

# Mount Google Drive
drive.flush_and_unmount()
!rm -rf /tmp/*

drive.mount("/content/gdrive", force_remount=True)
root_dir = "/content/gdrive/My Drive/dsai-2024/MVPS"
proj_dir = os.path.join(root_dir, 'Camp-Rock-Paper-Scissors')
os.chdir('/content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors')

rps_dir = os.path.join(proj_dir, 'gestures') # this points to data folder
train_dir = os.path.join(rps_dir, 'train')
test_dir = os.path.join(rps_dir, 'test')
model_dir = os.path.join(rps_dir, 'model')
base_dir = rps_dir

# Create directories for train, test, and model activities
for dir in ['train', 'test', 'model']:
    os.makedirs(os.path.join(rps_dir, dir), exist_ok=True)

# Define and create directories for classes
gestures = ['up', 'down', 'left', 'right', 'neutral']
for gesture in gestures:
    os.makedirs(os.path.join(train_dir, gesture), exist_ok=True)
    os.makedirs(os.path.join(test_dir, gesture), exist_ok=True)

# Record the end time and calculate the elapsed time
end_time = datetime.datetime.now()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time}")


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive
Elapsed time: 0:00:23.410330


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Capture and save images
for gesture in gestures:
    print(f"Capturing images for {gesture}. Press the 'Capture' button in the displayed UI.")
    for i in range(2):  # Adjust the range for more images
        filename = take_photo(os.path.join(base_dir, gesture, f'{gesture}_{i}.jpg'))
        print(f"Saved {filename}")


Capturing images for up. Press the 'Capture' button in the displayed UI.


<IPython.core.display.Javascript object>

MessageError: NotAllowedError: Permission denied

In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Preprocessing function
def load_data(base_dir, img_size=(128, 128)):
    X = []
    y = []
    gestures = ['up', 'down', 'left', 'right', 'neutral']
    label_map = {gesture: i for i, gesture in enumerate(gestures)}

    for gesture in gestures:
        gesture_dir = os.path.join(base_dir, gesture)
        for img_name in os.listdir(gesture_dir):
            img_path = os.path.join(gesture_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Read image in color mode
            if img is not None:
                img = cv2.resize(img, img_size)
                X.append(img)
                y.append(label_map[gesture])

    X = np.array(X, dtype='float32') / 255.0
    y = np.array(y, dtype='int')

    y = to_categorical(y, num_classes=len(gestures))

    return train_test_split(X, y, test_size=0.2, random_state=42)

# Load and split the data
X_train, X_test, y_train, y_test = load_data(base_dir)

# Load MobileNetV2 with pre-trained weights, exclude top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base_model layers so they are not trained
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
h = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Unfreeze the base_model layers and fine-tune the entire model
for layer in base_model.layers:
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
h = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Save the model
model.save(os.path.join(model_dir, 'mobilenetv2_head_gesture_model.h5'))


FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/up'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# Helper functions
def show_scores1(model, h, X_train, Y_train, X_test, Y_test):
    loss, acc = model.evaluate(X_train, Y_train, verbose=0)
    print ("Training: accuracy   = %.6f loss = %.6f" % (acc, loss))
    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    print ("Validation: accuracy = %.6f loss = %.6f" % (acc, loss))
    if 'val_accuracy' in h.history:
        print ("Over fitting score   = %.6f" % (max(h.history['accuracy']) - max(h.history['val_accuracy'])))
        print ("Under fitting score  = %.6f" % (max(h.history['val_accuracy']) - min(h.history['val_accuracy'])))
    print ("Params count:", model.count_params())
    print ("stop epoch =", max(h.epoch))
    print ("nb_epoch =", h.params['epochs'])
    view_acc1(h)
    plt.savefig('model_acc_graph.png')
    plt.show()
    view_loss(h)
    plt.savefig('model_loss_graph.png')
    plt.show()

def view_acc1(h):
    # Accuracy history graph
    plt.plot(h.history['accuracy'])
    if 'val_accuracy' in h.history:
        plt.plot(h.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    leg = plt.legend(['train', 'validation'], loc='best')
    plt.setp(leg.get_lines(), linewidth=3.0)

def view_loss(h):
    # Loss history graph
    plt.plot(h.history['loss'])
    if 'val_loss' in h.history:
        plt.plot(h.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    leg = plt.legend(['train', 'validation'], loc='best')
    plt.setp(leg.get_lines(), linewidth=3.0)

show_scores1(model, h, X_train, y_train, X_test, y_test)

# Evaluate the model using test data and generate detailed classification metrics
def evaluate_model(generator, model):
    all_labels = []
    all_preds = []
    for images, labels in generator:
        preds = model.predict(images)
        all_preds.extend(preds)
        all_labels.extend(labels)
        if len(all_labels) >= len(generator.labels):  # Prevent infinite loop
            break
    y_pred = np.argmax(np.array(all_preds), axis=1)
    y_true = np.argmax(np.array(all_labels), axis=1)
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=generator.class_indices.keys(), yticklabels=generator.class_indices.keys())
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=generator.class_indices.keys()))

Assuming test_generator is defined somewhere in the full script
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    os.path.join(rps_dir, 'test'),
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

evaluate_model(test_generator, model)
